In [ ]:
#優化版
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [ ]:

# 配置和初始化
PATH = "C:\\selenium\\msedgedriver.exe"  # 确保路径正确
course_path = "https://course.ntu.edu.tw/search/general"  # 替换为实际的网页地址
output_path = "courses.csv"  # 输出文件路径

service = Service(PATH)
driver = webdriver.Edge(service=service)

# 打开网页
driver.get(course_path)

# 等待页面加载并点击按钮（如果需要）
wait = WebDriverWait(driver, 10)
try:
    click_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[3]/div/div/div/button")))
    click_button.click()
except Exception as e:
    print(f"无法点击按钮: {e}")


In [ ]:

# 初始化变量
info_list = []  # 用于存储课程信息
last_height = driver.execute_script("return document.body.scrollHeight")  # 获取初始页面高度
# 初始化一个集合，用于存储已经抓取过的课程名称
seen_courses = set()

while True:
    # 等待页面加载完成
    courses = wait.until(EC.presence_of_all_elements_located((By.XPATH, "/html/body/div/main/div[6]/div/div[2]/ul/li")))

    # 遍历课程元素并抓取数据
    for course in courses:
        try:
            # 课程名称
            course_name = course.find_element(By.XPATH, ".//div[2]/h2/a").text.strip()
            # 老师
            teacher = course.find_element(By.XPATH, ".//div[2]/div/div[1]").text.strip()
            # 学分
            credits = course.find_element(By.XPATH, ".//div[3]/div[2]").text.strip()

            # 检查是否已经抓取过这个课程
            if course_name not in seen_courses:
                seen_courses.add(course_name)  # 将课程名称添加到集合中
                info_list.append({"課程名稱": course_name, "老師": teacher, "學分數": credits})
                print(f"課程名稱: {course_name}, 老師: {teacher}, 學分數: {credits}")
            else:
                print(f"课程 {course_name} 已经抓取过，跳过。")
        except Exception as e:
            print(f"无法获取课程信息: {e}")

    # 滚动页面到底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # 等待页面加载新内容

    # 检查是否滚动到底部
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("已经滚动到底部，停止滚动。")
        break
    last_height = new_height

In [ ]:
# 将数据保存为CSV文件
info_df = pd.DataFrame(info_list, columns=["課程名稱", "老師", "學分數"])
info_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"CSV文件已保存至: {output_path}")

# 关闭浏览器
driver.quit()